In [1]:
import pandas as pd

data = pd.read_excel("../all_transformations_ch_specialized_20240702_updated.xlsx")

In [2]:
print(data.columns)

def remove_atom_labels(input_synthon_smiles):
    synthon_copy = Chem.MolFromSmiles(input_synthon_smiles)
    for i in synthon_copy.GetAtoms():
        i.SetAtomMapNum(0)
    return Chem.MolToSmiles(synthon_copy)


Index(['input_target_molecule_smiles', 'mapped_input_target_molecule_smiles',
       'target_substructure_smarts', 'substructure_atom_map',
       'disconnection_map', 'synthon_a', 'synthon_a_label', 'synthon_b',
       'synthon_b_label', 'synthon_a_building_block_class',
       'synthon_a_building_block_label', 'synthon_b_building_block_class',
       'synthon_b_building_block_label', 'synthon_a_building_block_smiles',
       'synthon_a_building_block_cas', 'synthon_a_building_block_price',
       'synthon_a_building_block_material', 'synthon_b_building_block_smiles',
       'synthon_b_building_block_cas', 'synthon_b_building_block_price',
       'synthon_b_building_block_material', 'benzylic_a', 'alpha_het_a',
       'alpha_carbonyl_a', 'benzylic_b', 'alpha_het_b', 'alpha_carbonyl_b',
       'bond_formed', 'synth_class_specific_a', 'synth_class_specific_b',
       'synth_class_specific_a_updated', 'synth_class_specific_b_updated',
       'Unnamed: 32', 'benzylic_a_updated', 'benzylic

In [3]:
classes = ["aryl alcohol", "alkyl alcohol",
              "aryl amine", "alkyl amine",
              "aryl acid", "alkyl acid", 
              "aryl chloride", "alkyl chloride",
              "aryl bromide", "alkyl bromide", 
              "aryl iodide", "alkyl iodide", 
              "aryl boronate", "alkyl boronate", 
              "alpha_carbonyl", "alpha_allyl", "benzylic", "alpha_het", "other_hydrogen"]

In [5]:
bond_group = data.groupby("bond_formed")

In [6]:
for k in bond_group:
    print(k[0], len(k[1]))
    # if k[0] == "aryl-alkyl-cC":
    
        # print(k[0])
        # for i,j in k[1].iterrows():
            # print(j["synth_class_specific_b"])

alkyl-alkyl-CC 29658
alkyl-alkyl-CN 6741
alkyl-alkyl-CO 8462
alkyl-aryl-Cn 350
aryl-alkyl-cC 12505
aryl-alkyl-cN 2503
aryl-alkyl-cO 3501
aryl-aryl-cc 2484
aryl-aryl-cn 85


In [7]:
focusing = [
"alkyl-alkyl-CN",

"alkyl-alkyl-CO",

"alkyl-aryl-Cn",

"aryl-alkyl-cN",

"aryl-alkyl-cO",

"aryl-aryl-cn",
]

In [ ]:
from rdkit import Chem
import pycircos
import collections
import matplotlib.pyplot as plt
from matplotlib import cm
import matplotlib

cmap = cm.get_cmap("viridis")


for cl,dat in bond_group:
    if cl in focusing:
        continue

    for bb_class in classes:
        if bb_class in ["alpha_het", "benzylic", "alpha_allyl", "alpha_carbonyl"]:
            continue
        print(cl, bb_class)
        focus = bb_class

        unique_drugs = []
        a_synthons = []
        b_synthons = []
        a_drug_chords = []
        b_drug_chords = []
        a_b_chords = []
        a_class = []
        b_class = []

        syn_a_count = {}
        syn_b_count = {}

        for i,k in dat.iterrows():
            if k["synth_class_specific_a"] == focus:
                syn_a = k["synthon_a"]
                syn_b = k["synthon_b"]
                syn_a_sm = k["synthon_a_building_block_smiles"]
                syn_b_sm = k["synthon_b_building_block_smiles"]
                syn_a_class = k["synth_class_specific_a"]
                syn_b_class = k["synth_class_specific_b"]
            elif k["synth_class_specific_b"] == focus:
                syn_a = k["synthon_b"]
                syn_b = k["synthon_a"]
                syn_a_sm = k["synthon_b_building_block_smiles"]
                syn_b_sm = k["synthon_a_building_block_smiles"]
                syn_a_class = k["synth_class_specific_b"]
                syn_b_class = k["synth_class_specific_a"]
            else:
                continue
                
            if k["input_target_molecule_smiles"] not in unique_drugs:
                unique_drugs.append(k["input_target_molecule_smiles"])
                
            if syn_a_sm not in a_synthons:
                a_synthons.append(syn_a_sm)
                syn_a_count[syn_a_sm] = 0
            if syn_b_sm not in b_synthons:
                b_synthons.append(syn_b_sm)
                syn_b_count[syn_b_sm] = 0

            syn_a_count[syn_a_sm] = syn_a_count[syn_a_sm] + 1
            syn_b_count[syn_b_sm] = syn_b_count[syn_b_sm] + 1
            a_drug_chords.append([syn_a_sm, k["input_target_molecule_smiles"]])
            b_drug_chords.append([syn_b_sm, k["input_target_molecule_smiles"]])
            a_b_chords.append([syn_a_sm, syn_b_sm])
            a_class.append(syn_a_class)
            b_class.append(syn_b_class)


        print(len(unique_drugs), len(a_synthons), len(b_synthons), len(a_drug_chords), len(b_drug_chords), len(a_b_chords))
        # continue
        if len(unique_drugs) == 1:
            continue
        # continue
        if len(unique_drugs) == 0:
            continue

        if len(unique_drugs) < 150:
            corr = 1
        # elif len(unique_drugs) < 150:
            # corr = 2.5
        else:
            corr = 10
        a_class_set = list(set(a_class))
        b_class_set = list(set(b_class))



        Garc    = pycircos.Garc
        Gcircle = pycircos.Gcircle
        circle = Gcircle(figsize=(10,10)) 
        garc_lengths = []


        name   = "drugs"
        length = len(unique_drugs)/corr
        garc_lengths.append(length)
        arc    = Garc(arc_id=name, size=length, interspace=10, raxis_range=(950,960), labelposition=60, label_visible=False, \
                      facecolor='grey')

        circle.add_garc(arc) 

        name   = "a_synthons"
        length = len(a_synthons)
        garc_lengths.append(length)
        arc    = Garc(arc_id=name, size=length, interspace=10, raxis_range=(800,1100), labelposition=60, label_visible=False, \
                      facecolor='grey')
        circle.add_garc(arc) 

        name   = "b_synthons"
        length = len(b_synthons)
        garc_lengths.append(length)
        arc    = Garc(arc_id=name, size=length, interspace=10, raxis_range=(800,1100), labelposition=60, label_visible=False, \
                      facecolor='grey')
        circle.add_garc(arc) 

        circle.set_garcs() 



        arcdata_dict = collections.defaultdict(dict)
        # print(a_synthons)
        for i, synthon in enumerate(a_synthons):
            if "a_synthons" not in arcdata_dict:
                arcdata_dict["a_synthons"]["positions"] = []
                arcdata_dict["a_synthons"]["widths"]    = [] 
                arcdata_dict["a_synthons"]["colors"]    = [] 
            arcdata_dict["a_synthons"]["positions"].append(a_synthons.index(synthon)) 
            arcdata_dict["a_synthons"]["widths"].append(1)

            arcdata_dict["a_synthons"]["colors"].append(classes.index(a_class[i]))

            for key in arcdata_dict:
            # print(key, arcdata_dict[key])
                circle.heatmap(key, data=arcdata_dict[key]["colors"], positions=arcdata_dict[key]["positions"], 
                               width=arcdata_dict[key]["widths"], raxis_range=[800,1100], cmap=plt.cm.viridis, vmax = len(classes)-1, vmin = 0) 


        arcdata_dict = collections.defaultdict(dict)
        for i, synthon in enumerate(b_synthons):
            if "b_synthons" not in arcdata_dict:
                arcdata_dict["b_synthons"]["positions"] = []
                arcdata_dict["b_synthons"]["widths"]    = [] 
                arcdata_dict["b_synthons"]["colors"]    = [] 
            arcdata_dict["b_synthons"]["positions"].append(b_synthons.index(synthon)) 
            arcdata_dict["b_synthons"]["widths"].append(1)

            arcdata_dict["b_synthons"]["colors"].append(classes.index(b_class[i]))

            for key in arcdata_dict:
            # print(key, arcdata_dict[key])
                circle.heatmap(key, data=arcdata_dict[key]["colors"], positions=arcdata_dict[key]["positions"], 
                               width=arcdata_dict[key]["widths"], raxis_range=[800,1100], cmap=plt.cm.viridis, vmax = len(classes)-1, vmin = 0) 

        pair_info = []
        for i,chord in enumerate(a_drug_chords):
            r1loc = ((len(unique_drugs)-2) * unique_drugs.index(chord[1])/(len(unique_drugs)-1))/corr
            if len(a_synthons) == 2:
                r2loc = a_synthons.index(chord[0])/(2)
            else:
                r2loc = (len(a_synthons)-2) * a_synthons.index(chord[0])/(len(a_synthons)-1)
            source = ("drugs", r1loc, r1loc+1/corr, 800)
            if len(a_synthons) == 2:
                destination = ("a_synthons", r2loc+.2, r2loc+.3, 800)
            else:
                destination = ("a_synthons", r2loc+.4, r2loc+.6, 800)
            # print(r2loc)

            # col = 'def1de'
            # col =  tuple(int(col[i:i+2], 16)/255 for i in (0, 2, 4))
            col = cmap(syn_a_count[chord[0]]*10)
            pair_info.append((source, destination, tuple(col)))

            
        chord_color = (230/255, 231/255, 232/255, 0.5)

        for i,pair in enumerate(reversed(pair_info)):
            circle.chord_plot(pair[0], pair[1], facecolor=chord_color, linewidth=.01)



        pair_info = []
        for i,chord in enumerate(b_drug_chords):
            r1loc = ((len(unique_drugs)-2) * unique_drugs.index(chord[1])/(len(unique_drugs)-1))/corr
            r2loc = (len(b_synthons)-2) * b_synthons.index(chord[0])/(len(b_synthons)-1)
            source = ("drugs", r1loc, r1loc+1/corr, 800)
            destination = ("b_synthons", r2loc+.4, r2loc+.6, 800)

            # col = 'def1de'
            # col =  tuple(int(col[i:i+2], 16)/255 for i in (0, 2, 4))
            col = cmap(syn_b_count[chord[0]]*10)
            pair_info.append((source, destination, tuple(col)))

        for i,pair in enumerate(reversed(pair_info)):
            circle.chord_plot(pair[0], pair[1], facecolor=chord_color, linewidth=.01)



        # pair_info = []
        # for i,chord in enumerate(a_b_chords):
        #     r1loc = ((len(b_synthons)-2) * b_synthons.index(chord[1])/(len(b_synthons)-1))
        #     r2loc = (len(a_synthons)-2) * a_synthons.index(chord[0])/(len(a_synthons)-1)
        #     source = ("b_synthons", r1loc+.4, r1loc+.6, 950)
        #     destination = ("a_synthons", r2loc+.4, r2loc+.6, 950)

        #     col = '306030'
        #     col =  tuple(int(col[i:i+2], 16)/255 for i in (0, 2, 4))
        #     pair_info.append((source, destination, tuple(col)))

        # for i,pair in enumerate(reversed(pair_info)):
            # circle.chord_plot(pair[0], pair[1], facecolor=pair[2], linewidth=.01)

        circle.figure.savefig(f"./output_graphics2/{cl}-{focus}.png", dpi=300, transparent=True)


/var/folders/8k/w__0g4xs7rj0xv7hk3dqm71w0000gn/T/ipykernel_56591/4095515856.py:8: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cmap = cm.get_cmap("viridis")


alkyl-alkyl-CC aryl alcohol
0 0 0 0 0 0
alkyl-alkyl-CC alkyl alcohol
865 469 1552 7068 7068 7068


KeyboardInterrupt: 

In [89]:
import matplotlib.cm as cm

def get_viridis_color(range_min, range_max, value):
    colormap = cm.get_cmap('viridis')  # Get the Viridis color map
    normed_value = (value - range_min) / (range_max - range_min)  # Normalize the value

    # Get the RGB color tuple from the color map
    rgb_color = colormap(normed_value)

    # Convert the RGB values to hexadecimal
    hex_color = '#{:02x}{:02x}{:02x}'.format(int(rgb_color[0]*255), int(rgb_color[1]*255), int(rgb_color[2]*255))

    return hex_color

In [72]:
for bb_class in classes:
    
    ind = classes.index(bb_class)
    mi = 0
    ma = len(classes) - 1
    
    print(bb_class, get_viridis_color(mi, ma, ind))

aryl alcohol #440154
alkyl alcohol #471567
aryl amine #472777
alkyl amine #443982
aryl acid #3e4989
alkyl acid #37598c
aryl chloride #30678d
alkyl chloride #2a758e
aryl bromide #25828e
alkyl bromide #20908c
aryl iodide #1e9d88
alkyl iodide #24aa82
aryl boronate #35b778
alkyl boronate #4dc26b
alpha_carbonyl #6dce58
alpha_allyl #90d643
benzylic #b5dd2b
alpha_het #dae218
other_hydrogen #fde724


In [47]:
syn_a = []
syn_b = []
# print(k2.columns)
for k, k2 in bond_group:
    if k == "aryl-aryl-cc":
        for i,row in k2.iterrows():
            if row["synth_class_specific_b"] == "aryl iodide":
                # print(row["synthon_b"])
                syn_a.append(row["synthon_a_building_block_smiles"])
                syn_b.append(row["synthon_b_building_block_smiles"])

print(len(set(syn_a)))   
print(len(set(syn_b)))    
print(set(syn_b))
        # out = k2.groupby("synth_class_specific_b")
        # for j, j2 in out:
            # print(j)
                # print(j2)
                # out2 = j2.groupby("synthon_b")
                # for i, i2 in out2:
                    # print(i)
                # print(j2)

34
2
{'Ic1cnc[nH]1', 'Ic1cc[nH]n1'}


In [75]:
def is_alpha_het_carbon(mol, carbon_atom_label):
    hits = mol.GetSubstructMatches(alpha_het_sub)
    for k in hits:
        for j in k:
            atom = mol.GetAtomWithIdx(j)
            if atom.GetAtomMapNum() == int(carbon_atom_label):
                return True
    return False


In [77]:
alpha_het_sub = Chem.MolFromSmarts("[CX4H2:3][N,S,O]")

alpha_het_b = []
for i,k in data.iterrows():
    syn_b_mol = Chem.MolFromSmiles(k["synthon_b"])

    alpha_het = is_alpha_het_carbon(syn_b_mol,k["synthon_b_label"])
    alpha_het_b.append(alpha_het)
print(data.columns)

Index(['Unnamed: 0', 'Unnamed: 0.1', 'input_target_molecule_smiles',
       'mapped_input_target_molecule_smiles', 'target_substructure_smarts',
       'substructure_atom_map', 'disconnection_map', 'synthon_a',
       'synthon_a_label', 'synthon_b', 'synthon_b_label',
       'synthon_a_building_block_class', 'synthon_a_building_block_label',
       'synthon_b_building_block_class', 'synthon_b_building_block_label',
       'synthon_a_building_block_smiles', 'synthon_a_building_block_cas',
       'synthon_a_building_block_price', 'synthon_a_building_block_material',
       'synthon_b_building_block_smiles', 'synthon_b_building_block_cas',
       'synthon_b_building_block_price', 'synthon_b_building_block_material',
       'aromatic_a', 'benzylic_a', 'alpha_het_a', 'alpha_allyl_a',
       'alpha_carbonyl_a', 'aromatic_b', 'benzylic_b', 'alpha_het_b',
       'alpha_allyl_b', 'alpha_carbonyl_b', 'bond_formed',
       'synth_class_specific_a', 'synth_class_specific_b'],
      dtype='object')

In [102]:
updated_synth_class_specific_b = []

for i,k in data.iterrows():
    if k["alpha_het_b"]:
        if k["synth_class_specific_b"] == "other_hydrogen":
            updated_synth_class_specific_b.append("alpha_het")
            continue
    updated_synth_class_specific_b.append(k["synth_class_specific_b"])

In [103]:
data["synth_class_specific_b"] = updated_synth_class_specific_b

In [104]:
data.to_excel("all_transformation_data.xlsx")

In [6]:
import pandas as pd

data = pd.read_excel("output_datasets/aryl-alkyl-Cc.xlsx")

In [ ]:
from rdkit import Chem
import pycircos
import collections
import matplotlib.pyplot as plt
from matplotlib import cm
import matplotlib

cmap = cm.get_cmap("viridis")

for k in classes:
    print(k)
    focus = k

    unique_drugs = []
    a_synthons = []
    b_synthons = []
    a_drug_chords = []
    b_drug_chords = []
    a_b_chords = []
    a_class = []
    b_class = []

    syn_a_count = {}
    syn_b_count = {}

    for i,k in data.iterrows():
        if k["synthon_a_building_block_class"] != focus:
            continue
        if k["input_target_molecule_smiles"] not in unique_drugs:
            unique_drugs.append(k["input_target_molecule_smiles"])
        syn_a = remove_atom_labels(k["synthon_a"])
        syn_b = remove_atom_labels(k["synthon_b"])
        if k["synthon_a_building_block_smiles"] not in a_synthons:
            a_synthons.append(k["synthon_a_building_block_smiles"])
            syn_a_count[k["synthon_a_building_block_smiles"]] = 0
        if k["synthon_b_building_block_smiles"] not in b_synthons:
            b_synthons.append(k["synthon_b_building_block_smiles"])
            syn_b_count[k["synthon_b_building_block_smiles"]] = 0

        syn_a_count[k["synthon_a_building_block_smiles"]] = syn_a_count[k["synthon_a_building_block_smiles"]] + 1
        syn_b_count[k["synthon_b_building_block_smiles"]] = syn_b_count[k["synthon_b_building_block_smiles"]] + 1
        a_drug_chords.append([k["synthon_a_building_block_smiles"], k["input_target_molecule_smiles"]])
        b_drug_chords.append([k["synthon_b_building_block_smiles"], k["input_target_molecule_smiles"]])
        a_b_chords.append([k["synthon_a_building_block_smiles"], k["synthon_b_building_block_smiles"]])
        a_class.append(k["synthon_a_building_block_class"])
        b_class.append(k["synthon_b_building_block_class"])


    print(len(unique_drugs), len(a_synthons), len(b_synthons), len(a_drug_chords), len(b_drug_chords), len(a_b_chords))
    if len(unique_drugs) == 0:
        continue
        
    if len(unique_drugs) < 100:
        corr = 1
    else:
        corr = 10
    a_class_set = list(set(a_class))
    b_class_set = list(set(b_class))



    Garc    = pycircos.Garc
    Gcircle = pycircos.Gcircle
    circle = Gcircle(figsize=(10,10)) 
    garc_lengths = []


    name   = "drugs"
    length = len(unique_drugs)/corr
    garc_lengths.append(length)
    arc    = Garc(arc_id=name, size=length, interspace=10, raxis_range=(950,960), labelposition=60, label_visible=False, \
                  facecolor='grey')

    circle.add_garc(arc) 

    name   = "a_synthons"
    length = len(a_synthons)
    garc_lengths.append(length)
    arc    = Garc(arc_id=name, size=length, interspace=10, raxis_range=(950,1100), labelposition=60, label_visible=False, \
                  facecolor='grey')
    circle.add_garc(arc) 

    name   = "b_synthons"
    length = len(b_synthons)
    garc_lengths.append(length)
    arc    = Garc(arc_id=name, size=length, interspace=10, raxis_range=(950,1100), labelposition=60, label_visible=False, \
                  facecolor='grey')
    circle.add_garc(arc) 

    circle.set_garcs() 



    arcdata_dict = collections.defaultdict(dict)
    for i, synthon in enumerate(a_synthons):
        if "a_synthons" not in arcdata_dict:
            arcdata_dict["a_synthons"]["positions"] = []
            arcdata_dict["a_synthons"]["widths"]    = [] 
            arcdata_dict["a_synthons"]["colors"]    = [] 
        arcdata_dict["a_synthons"]["positions"].append(a_synthons.index(synthon)) 
        arcdata_dict["a_synthons"]["widths"].append(1)

        arcdata_dict["a_synthons"]["colors"].append(classes.index(a_class[i]))

        for key in arcdata_dict:
        # print(key, arcdata_dict[key])
            circle.heatmap(key, data=arcdata_dict[key]["colors"], positions=arcdata_dict[key]["positions"], 
                           width=arcdata_dict[key]["widths"], raxis_range=[950,1100], cmap=plt.cm.viridis, vmax = 7, vmin = 0) 


    arcdata_dict = collections.defaultdict(dict)
    for i, synthon in enumerate(b_synthons):
        if "b_synthons" not in arcdata_dict:
            arcdata_dict["b_synthons"]["positions"] = []
            arcdata_dict["b_synthons"]["widths"]    = [] 
            arcdata_dict["b_synthons"]["colors"]    = [] 
        arcdata_dict["b_synthons"]["positions"].append(b_synthons.index(synthon)) 
        arcdata_dict["b_synthons"]["widths"].append(1)

        arcdata_dict["b_synthons"]["colors"].append(classes.index(b_class[i]))

        for key in arcdata_dict:
        # print(key, arcdata_dict[key])
            circle.heatmap(key, data=arcdata_dict[key]["colors"], positions=arcdata_dict[key]["positions"], 
                           width=arcdata_dict[key]["widths"], raxis_range=[950,1100], cmap=plt.cm.viridis, vmax = 7, vmin = 0) 

    pair_info = []
    for i,chord in enumerate(a_drug_chords):
        r1loc = ((len(unique_drugs)-2) * unique_drugs.index(chord[1])/(len(unique_drugs)-1))/corr
        r2loc = (len(a_synthons)-2) * a_synthons.index(chord[0])/(len(a_synthons)-1)
        source = ("drugs", r1loc, r1loc+1/corr, 950)
        destination = ("a_synthons", r2loc+.4, r2loc+.6, 950)

        # col = 'def1de'
        # col =  tuple(int(col[i:i+2], 16)/255 for i in (0, 2, 4))
        col = cmap(syn_a_count[chord[0]]*10)
        pair_info.append((source, destination, tuple(col)))

    for i,pair in enumerate(reversed(pair_info)):
        circle.chord_plot(pair[0], pair[1], facecolor=pair[2], linewidth=.01)



    pair_info = []
    for i,chord in enumerate(b_drug_chords):
        r1loc = ((len(unique_drugs)-2) * unique_drugs.index(chord[1])/(len(unique_drugs)-1))/corr
        r2loc = (len(b_synthons)-2) * b_synthons.index(chord[0])/(len(b_synthons)-1)
        source = ("drugs", r1loc, r1loc+1/corr, 950)
        destination = ("b_synthons", r2loc+.4, r2loc+.6, 950)

        # col = 'def1de'
        # col =  tuple(int(col[i:i+2], 16)/255 for i in (0, 2, 4))
        col = cmap(syn_b_count[chord[0]]*10)
        pair_info.append((source, destination, tuple(col)))

    for i,pair in enumerate(reversed(pair_info)):
        circle.chord_plot(pair[0], pair[1], facecolor=pair[2], linewidth=.01)



    # pair_info = []
    # for i,chord in enumerate(a_b_chords):
    #     r1loc = ((len(b_synthons)-2) * b_synthons.index(chord[1])/(len(b_synthons)-1))
    #     r2loc = (len(a_synthons)-2) * a_synthons.index(chord[0])/(len(a_synthons)-1)
    #     source = ("b_synthons", r1loc+.4, r1loc+.6, 950)
    #     destination = ("a_synthons", r2loc+.4, r2loc+.6, 950)

    #     col = '306030'
    #     col =  tuple(int(col[i:i+2], 16)/255 for i in (0, 2, 4))
    #     pair_info.append((source, destination, tuple(col)))

    # for i,pair in enumerate(reversed(pair_info)):
        # circle.chord_plot(pair[0], pair[1], facecolor=pair[2], linewidth=.01)

    circle.figure.savefig(f"output_graphics/aryl-alkyl-Cc-{focus}.png", dpi=300, transparent=True)
# print(len(unique_drugs), len(a_synthons), len(b_synthons), len(a_drug_chords), len(b_drug_chords), len(a_b_chords))


alcohol
531 162 652 1794 1794 1794
chloride
539 166 650 1851 1851 1851
acid
536 166 636 1775 1775 1775
amine
522 172 648 1746 1746 1746
iodide
15 5 39 55 55 55
bromide
411 97 554 1343 1343 1343
hydrogen
712 277 830 2728 2728 2728
boronate
402 96 507 1213 1213 1213


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Error in callback <function flush_figures at 0x128620040> (for post_execute):
Traceback (most recent call last):
  File "/opt/homebrew/Caskroom/miniforge/base/envs/sklearn-env/lib/python3.9/site-packages/IPython/core/events.py", line 89, in trigger
    func(*args, **kwargs)
  File "/opt/homebrew/Caskroom/miniforge/base/envs/sklearn-env/lib/python3.9/site-packages/matplotlib_inline/backend_inline.py", line 121, in flush_figures
    return show(True)
  File "/opt/homebrew/Caskroom/miniforge/base/envs/sklearn-env/lib/python3.9/site-packages/matplotlib_inline/backend_inline.py", line 41, in show
    display(
  File "/opt/homebrew/Caskroom/miniforge/base/envs/sklearn-env/lib/python3.9/site-packages/IPython/core/display.py", line 320, in display
    format_dict, md_dict = format(obj, include=include, exclude=exclude)
  File "/opt/homebrew/Caskroom/miniforge/base/envs/sklearn-env/lib/python3.9/site-packages/IPython/core/formatters.py", line 180, in format
    data = formatter(obj)
  File "/op

In [8]:
import pandas as pd

data = pd.read_excel("output_datasets/alkyl-alkyl-CC.xlsx")

In [ ]:
from rdkit import Chem
import pycircos
import collections
import matplotlib.pyplot as plt
from matplotlib import cm
import matplotlib

cmap = cm.get_cmap("viridis")

for k in classes:
    print(k)
    focus = k

    unique_drugs = []
    a_synthons = []
    b_synthons = []
    a_drug_chords = []
    b_drug_chords = []
    a_b_chords = []
    a_class = []
    b_class = []

    syn_a_count = {}
    syn_b_count = {}

    for i,k in data.iterrows():
        if k["synthon_a_building_block_class"] != focus:
            continue
        if k["input_target_molecule_smiles"] not in unique_drugs:
            unique_drugs.append(k["input_target_molecule_smiles"])
        syn_a = remove_atom_labels(k["synthon_a"])
        syn_b = remove_atom_labels(k["synthon_b"])
        if k["synthon_a_building_block_smiles"] not in a_synthons:
            a_synthons.append(k["synthon_a_building_block_smiles"])
            syn_a_count[k["synthon_a_building_block_smiles"]] = 0
        if k["synthon_b_building_block_smiles"] not in b_synthons:
            b_synthons.append(k["synthon_b_building_block_smiles"])
            syn_b_count[k["synthon_b_building_block_smiles"]] = 0

        syn_a_count[k["synthon_a_building_block_smiles"]] = syn_a_count[k["synthon_a_building_block_smiles"]] + 1
        syn_b_count[k["synthon_b_building_block_smiles"]] = syn_b_count[k["synthon_b_building_block_smiles"]] + 1
        a_drug_chords.append([k["synthon_a_building_block_smiles"], k["input_target_molecule_smiles"]])
        b_drug_chords.append([k["synthon_b_building_block_smiles"], k["input_target_molecule_smiles"]])
        a_b_chords.append([k["synthon_a_building_block_smiles"], k["synthon_b_building_block_smiles"]])
        a_class.append(k["synthon_a_building_block_class"])
        b_class.append(k["synthon_b_building_block_class"])


    print(len(unique_drugs), len(a_synthons), len(b_synthons), len(a_drug_chords), len(b_drug_chords), len(a_b_chords))
    if len(unique_drugs) == 0:
        continue
        
    if len(unique_drugs) < 100:
        corr = 1
    else:
        corr = 10
    a_class_set = list(set(a_class))
    b_class_set = list(set(b_class))



    Garc    = pycircos.Garc
    Gcircle = pycircos.Gcircle
    circle = Gcircle(figsize=(10,10)) 
    garc_lengths = []


    name   = "drugs"
    length = len(unique_drugs)/corr
    garc_lengths.append(length)
    arc    = Garc(arc_id=name, size=length, interspace=10, raxis_range=(950,960), labelposition=60, label_visible=False, \
                  facecolor='grey')

    circle.add_garc(arc) 

    name   = "a_synthons"
    length = len(a_synthons)
    garc_lengths.append(length)
    arc    = Garc(arc_id=name, size=length, interspace=10, raxis_range=(950,1100), labelposition=60, label_visible=False, \
                  facecolor='grey')
    circle.add_garc(arc) 

    name   = "b_synthons"
    length = len(b_synthons)
    garc_lengths.append(length)
    arc    = Garc(arc_id=name, size=length, interspace=10, raxis_range=(950,1100), labelposition=60, label_visible=False, \
                  facecolor='grey')
    circle.add_garc(arc) 

    circle.set_garcs() 



    arcdata_dict = collections.defaultdict(dict)
    for i, synthon in enumerate(a_synthons):
        if "a_synthons" not in arcdata_dict:
            arcdata_dict["a_synthons"]["positions"] = []
            arcdata_dict["a_synthons"]["widths"]    = [] 
            arcdata_dict["a_synthons"]["colors"]    = [] 
        arcdata_dict["a_synthons"]["positions"].append(a_synthons.index(synthon)) 
        arcdata_dict["a_synthons"]["widths"].append(1)

        arcdata_dict["a_synthons"]["colors"].append(classes.index(a_class[i]))

        for key in arcdata_dict:
        # print(key, arcdata_dict[key])
            circle.heatmap(key, data=arcdata_dict[key]["colors"], positions=arcdata_dict[key]["positions"], 
                           width=arcdata_dict[key]["widths"], raxis_range=[950,1100], cmap=plt.cm.viridis, vmax = 7, vmin = 0) 


    arcdata_dict = collections.defaultdict(dict)
    for i, synthon in enumerate(b_synthons):
        if "b_synthons" not in arcdata_dict:
            arcdata_dict["b_synthons"]["positions"] = []
            arcdata_dict["b_synthons"]["widths"]    = [] 
            arcdata_dict["b_synthons"]["colors"]    = [] 
        arcdata_dict["b_synthons"]["positions"].append(b_synthons.index(synthon)) 
        arcdata_dict["b_synthons"]["widths"].append(1)

        arcdata_dict["b_synthons"]["colors"].append(classes.index(b_class[i]))

        for key in arcdata_dict:
        # print(key, arcdata_dict[key])
            circle.heatmap(key, data=arcdata_dict[key]["colors"], positions=arcdata_dict[key]["positions"], 
                           width=arcdata_dict[key]["widths"], raxis_range=[950,1100], cmap=plt.cm.viridis, vmax = 7, vmin = 0) 

    pair_info = []
    for i,chord in enumerate(a_drug_chords):
        r1loc = ((len(unique_drugs)-2) * unique_drugs.index(chord[1])/(len(unique_drugs)-1))/corr
        r2loc = (len(a_synthons)-2) * a_synthons.index(chord[0])/(len(a_synthons)-1)
        source = ("drugs", r1loc, r1loc+1/corr, 950)
        destination = ("a_synthons", r2loc+.4, r2loc+.6, 950)

        # col = 'def1de'
        # col =  tuple(int(col[i:i+2], 16)/255 for i in (0, 2, 4))
        col = cmap(syn_a_count[chord[0]]*10)
        pair_info.append((source, destination, tuple(col)))

    for i,pair in enumerate(reversed(pair_info)):
        circle.chord_plot(pair[0], pair[1], facecolor=pair[2], linewidth=.01)



    pair_info = []
    for i,chord in enumerate(b_drug_chords):
        r1loc = ((len(unique_drugs)-2) * unique_drugs.index(chord[1])/(len(unique_drugs)-1))/corr
        r2loc = (len(b_synthons)-2) * b_synthons.index(chord[0])/(len(b_synthons)-1)
        source = ("drugs", r1loc, r1loc+1/corr, 950)
        destination = ("b_synthons", r2loc+.4, r2loc+.6, 950)

        # col = 'def1de'
        # col =  tuple(int(col[i:i+2], 16)/255 for i in (0, 2, 4))
        col = cmap(syn_b_count[chord[0]]*10)
        pair_info.append((source, destination, tuple(col)))

    for i,pair in enumerate(reversed(pair_info)):
        circle.chord_plot(pair[0], pair[1], facecolor=pair[2], linewidth=.01)



    # pair_info = []
    # for i,chord in enumerate(a_b_chords):
    #     r1loc = ((len(b_synthons)-2) * b_synthons.index(chord[1])/(len(b_synthons)-1))
    #     r2loc = (len(a_synthons)-2) * a_synthons.index(chord[0])/(len(a_synthons)-1)
    #     source = ("b_synthons", r1loc+.4, r1loc+.6, 950)
    #     destination = ("a_synthons", r2loc+.4, r2loc+.6, 950)

    #     col = '306030'
    #     col =  tuple(int(col[i:i+2], 16)/255 for i in (0, 2, 4))
    #     pair_info.append((source, destination, tuple(col)))

    # for i,pair in enumerate(reversed(pair_info)):
        # circle.chord_plot(pair[0], pair[1], facecolor=pair[2], linewidth=.01)

    circle.figure.savefig(f"output_graphics/alkyl-alkyl-CC-{focus}.png", dpi=300, transparent=True)
    # print(len(unique_drugs), len(a_synthons), len(b_synthons), len(a_drug_chords), len(b_drug_chords), len(a_b_chords))


alcohol
645 277 1139 3718 3718 3718
chloride
526 159 1035 2874 2874 2874
acid
983 431 1540 5492 5492 5492
amine
672 248 1159 3734 3734 3734
iodide
370 32 768 1594 1594 1594
bromide
458 104 955 2400 2400 2400
hydrogen
1276 780 1825 8105 8105 8105
boronate
355 25 809 1741 1741 1741


In [17]:
matplotlib.colors.rgb2hex(cmap(7*(1/7)))

'#fde725'

In [87]:
len(b_class_set)

8

In [88]:
b_class_set

['alcohol',
 'chloride',
 'acid',
 'amine',
 'iodide',
 'bromide',
 'hydrogen',
 'boronate']

In [157]:
import pandas as pd

data = pd.read_excel("output_datasets/alkyl-aryl-Cn.xlsx")

In [164]:
benz = Chem.MolFromSmarts("[c][CX4H2][N]")
for i,k in data.iterrows():
    mol = Chem.MolFromSmiles(k["mapped_input_target_molecule_smiles"])
    hits = mol.GetSubstructMatches(benz)
    
    for i in hits:
        # first = True
        # for j in i:
        #     if first:
        #         first = False
        #         continue
        # n = mol.GetAtomWithIdx(j).GetAtomMapNum()
        # if n == 1 or n == 2:
        print(k)
        

Unnamed: 0                                                                     0
Unnamed: 0.1                                                                   0
input_target_molecule_smiles                     CN1CCc2c(c3ccccc3n2Cc2ccccc2)C1
mapped_input_target_molecule_smiles    CN1CCc2c(c3ccccc3[n:2]2[CH2:1]c2ccccc2)C1
target_substructure_smarts                                           [C:1]-[n:2]
substructure_atom_map                                                     [1, 2]
disconnection_map                                                         [1, 2]
synthon_a                                                      c1ccc([CH3:1])cc1
synthon_a_label                                                                1
synthon_b                                            CN1CCc2c(c3ccccc3[nH:2]2)C1
synthon_b_label                                                                2
synthon_a_building_block_class                                          hydrogen
synthon_a_building_block_lab